In [ ]:
!pip install faiss-cpu sentence-transformers

# Rag.py

In [ ]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

DATA_PATH = "/content/rag_chunks_engineered_v2.jsonl"
INDEX_PATH = "faiss_index.bin"
METADATA_PATH = "metadata.json"

model = SentenceTransformer("all-MiniLM-L6-v2")

documents = []
metadata = []

with open(DATA_PATH, "r") as f:
    for line in f:
        obj = json.loads(line)

        # Filter only table_of_cover if desired
        if obj["doc_type"] == "table_of_cover":
            documents.append(obj["text"])
            metadata.append(obj)

embeddings = model.encode(documents, show_progress_bar=True)
embeddings = np.array(embeddings).astype("float32")

# Normalize for cosine similarity
faiss.normalize_L2(embeddings)

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

faiss.write_index(index, INDEX_PATH)

with open(METADATA_PATH, "w") as f:
    json.dump(metadata, f)

print("FAISS index built successfully.")

def search(query, k=3):
    query_embedding = model.encode([query])
    query_embedding = np.array(query_embedding).astype("float32")
    faiss.normalize_L2(query_embedding)

    scores, indices = index.search(query_embedding, k)

    results = []
    for idx in indices[0]:
        results.append(metadata[idx])

    return results


In [ ]:
query = """
Coverage for cardiac treatment, frequent hospital visits,
consultant fees, medication support,
and minimal excess for inpatient admission.
"""

results = search(query)
for r in results:
    print(r["plan_name"])
    print(r["text"][:500])
    print("="*80)

# Evaluation.py

In [ ]:
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# ----------------------------
# CONFIG
# ----------------------------

INDEX_PATH = "faiss_index.bin"
METADATA_PATH = "metadata.json"
TOP_K = 3

# ----------------------------
# LOAD MODEL + INDEX
# ----------------------------

print("Loading embedding model...")
model = SentenceTransformer("all-MiniLM-L6-v2")

print("Loading FAISS index...")
index = faiss.read_index(INDEX_PATH)

print("Loading metadata...")
with open(METADATA_PATH, "r") as f:
    metadata = json.load(f)

# ----------------------------
# SEARCH FUNCTION
# ----------------------------

def search(query, k=TOP_K):
    query_embedding = model.encode([query])
    query_embedding = np.array(query_embedding).astype("float32")

    # Normalize query for cosine similarity
    faiss.normalize_L2(query_embedding)

    scores, indices = index.search(query_embedding, k)

    return [metadata[idx] for idx in indices[0]]


# ----------------------------
# AUTO DOC_ID RESOLUTION
# ----------------------------

def get_doc_id(plan_keyword):
    matches = [m["doc_id"] for m in metadata if plan_keyword.lower() in m["plan_name"].lower()]
    if not matches:
        raise ValueError(f"No doc_id found for keyword: {plan_keyword}")
    return matches[0]  # assume unique plan names

# ----------------------------
# BUILD EVALUATION SET
# ----------------------------

evaluation_set = [
    {
        "query": "Does Horizon 4 cover inpatient consultant fees?",
        "expected_doc_id": get_doc_id("Horizon 4")
    },
    {
        "query": "Which plan has a €300 excess for semi-private room admission?",
        "expected_doc_id": get_doc_id("300")
    },
    {
        "query": "How many days of psychiatric treatment are covered under Plan A?",
        "expected_doc_id": get_doc_id("Plan A")
    },
    {
        "query": "Are inpatient scans fully covered under Health Plan 26.1?",
        "expected_doc_id": get_doc_id("26.1")
    }
]

# ----------------------------
# METRIC COMPUTATION
# ----------------------------

correct_top1 = 0
correct_topk = 0
reciprocal_ranks = []

print("\nRunning evaluation...\n")

for item in evaluation_set:
    query = item["query"]
    expected = item["expected_doc_id"]

    results = search(query)
    returned_ids = [r["doc_id"] for r in results]

    print("Query:", query)
    print("Expected:", expected)
    print("Returned:", returned_ids)
    print("-" * 60)

    if returned_ids[0] == expected:
        correct_top1 += 1

    if expected in returned_ids:
        correct_topk += 1
        rank = returned_ids.index(expected) + 1
        reciprocal_ranks.append(1.0 / rank)
    else:
        reciprocal_ranks.append(0.0)

# ----------------------------
# FINAL METRICS
# ----------------------------

total = len(evaluation_set)

top1_accuracy = correct_top1 / total
topk_recall = correct_topk / total
mrr = sum(reciprocal_ranks) / total

print("\n===== FINAL METRICS =====")
print(f"Top-1 Accuracy: {top1_accuracy:.3f}")
print(f"Top-{TOP_K} Recall: {topk_recall:.3f}")
print(f"MRR: {mrr:.3f}")


# Phase 3 Clean BM25 Baseline Code: evaluate_bm25.py

In [ ]:
! pip install rank-bm25

In [ ]:
import json
import numpy as np
import re
from rank_bm25 import BM25Okapi

# ----------------------------
# CONFIG
# ----------------------------

METADATA_PATH = "metadata.json"
TOP_K = 3

# ----------------------------
# LOAD + FILTER METADATA
# ----------------------------

print("Loading metadata...")
with open(METADATA_PATH, "r") as f:
    metadata = json.load(f)

# Keep only Table of Cover documents
filtered_metadata = [
    m for m in metadata
    if m["doc_type"] == "table_of_cover"
]

print(f"Using {len(filtered_metadata)} table_of_cover documents")

# ----------------------------
# TOKENIZATION
# ----------------------------

def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9€]+", " ", text)
    return text.split()

corpus = [m["text"] for m in filtered_metadata]
tokenized_corpus = [tokenize(doc) for doc in corpus]

print("Building BM25 index...")
bm25 = BM25Okapi(tokenized_corpus)

# ----------------------------
# SEARCH
# ----------------------------

def search_bm25(query, k=TOP_K):
    tokenized_query = tokenize(query)
    scores = bm25.get_scores(tokenized_query)
    top_indices = np.argsort(scores)[::-1][:k]
    return [filtered_metadata[i] for i in top_indices]

# ----------------------------
# DOC ID RESOLUTION
# ----------------------------

def get_doc_id(plan_keyword):
    matches = [
        m["doc_id"]
        for m in filtered_metadata
        if plan_keyword.lower() in m["plan_name"].lower()
    ]
    return matches[0]

# ----------------------------
# EVALUATION SET
# ----------------------------

evaluation_set = [
    {
        "query": "Does Horizon 4 cover inpatient consultant fees?",
        "expected_doc_id": get_doc_id("Horizon 4")
    },
    {
        "query": "Which plan has a €300 excess for semi-private room admission?",
        "expected_doc_id": get_doc_id("300")
    },
    {
        "query": "How many days of psychiatric treatment are covered under Plan A?",
        "expected_doc_id": get_doc_id("Plan A")
    },
    {
        "query": "Are inpatient scans fully covered under Health Plan 26.1?",
        "expected_doc_id": get_doc_id("26.1")
    }
]

# ----------------------------
# METRICS
# ----------------------------

correct_top1 = 0
correct_topk = 0
reciprocal_ranks = []

print("\nRunning BM25 (clean) evaluation...\n")

for item in evaluation_set:
    query = item["query"]
    expected = item["expected_doc_id"]

    results = search_bm25(query)
    returned_ids = [r["doc_id"] for r in results]

    print("Query:", query)
    print("Returned:", returned_ids)
    print("-" * 60)

    if returned_ids[0] == expected:
        correct_top1 += 1

    if expected in returned_ids:
        correct_topk += 1
        rank = returned_ids.index(expected) + 1
        reciprocal_ranks.append(1.0 / rank)
    else:
        reciprocal_ranks.append(0.0)

total = len(evaluation_set)

print("\n===== BM25 (FILTERED) =====")
print("Top-1:", correct_top1 / total)
print("Top-3:", correct_topk / total)
print("MRR:", sum(reciprocal_ranks) / total)


# Hybrid Strategy.py

In [ ]:
import json
import numpy as np
import faiss
import re
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer

# ----------------------------
# CONFIG
# ----------------------------

INDEX_PATH = "faiss_index.bin"
METADATA_PATH = "metadata.json"
TOP_K = 3
ALPHA = 0.6
BETA = 0.4

# ----------------------------
# LOAD DATA
# ----------------------------

print("Loading model...")
model = SentenceTransformer("all-MiniLM-L6-v2")

print("Loading FAISS index...")
index = faiss.read_index(INDEX_PATH)

print("Loading metadata...")
with open(METADATA_PATH, "r") as f:
    metadata = json.load(f)

# Filter to table_of_cover only
filtered_indices = [
    i for i, m in enumerate(metadata)
    if m["doc_type"] == "table_of_cover"
]

filtered_metadata = [metadata[i] for i in filtered_indices]

print(f"Using {len(filtered_metadata)} table_of_cover documents")

# ----------------------------
# BM25 SETUP
# ----------------------------

def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9€]+", " ", text)
    return text.split()

corpus = [m["text"] for m in filtered_metadata]
tokenized_corpus = [tokenize(doc) for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

# ----------------------------
# HYBRID SEARCH
# ----------------------------

def normalize(scores):
    min_s = np.min(scores)
    max_s = np.max(scores)
    if max_s - min_s == 0:
        return np.zeros_like(scores)
    return (scores - min_s) / (max_s - min_s)

def hybrid_search(query, k=TOP_K):

    # Dense scores for all docs
    query_embedding = model.encode([query])
    query_embedding = np.array(query_embedding).astype("float32")

    dense_scores_all, _ = index.search(query_embedding, len(metadata))
    dense_scores_all = -dense_scores_all[0]

    # Filter dense scores
    dense_scores = np.array([dense_scores_all[i] for i in filtered_indices])

    # BM25 scores
    tokenized_query = tokenize(query)
    bm25_scores = bm25.get_scores(tokenized_query)

    # Normalize
    dense_norm = normalize(dense_scores)
    bm25_norm = normalize(bm25_scores)

    # Fusion
    final_scores = ALPHA * dense_norm + BETA * bm25_norm

    top_indices = np.argsort(final_scores)[::-1][:k]
    return [filtered_metadata[i] for i in top_indices]

# ----------------------------
# EVALUATION
# ----------------------------

def get_doc_id(plan_keyword):
    matches = [
        m["doc_id"]
        for m in filtered_metadata
        if plan_keyword.lower() in m["plan_name"].lower()
    ]
    return matches[0]

evaluation_set = [
    {
        "query": "Does Horizon 4 cover inpatient consultant fees?",
        "expected_doc_id": get_doc_id("Horizon 4")
    },
    {
        "query": "Which plan has a €300 excess for semi-private room admission?",
        "expected_doc_id": get_doc_id("300")
    },
    {
        "query": "How many days of psychiatric treatment are covered under Plan A?",
        "expected_doc_id": get_doc_id("Plan A")
    },
    {
        "query": "Are inpatient scans fully covered under Health Plan 26.1?",
        "expected_doc_id": get_doc_id("26.1")
    }
]

correct_top1 = 0
correct_topk = 0
reciprocal_ranks = []

print("\nRunning CLEAN Hybrid evaluation...\n")

for item in evaluation_set:
    query = item["query"]
    expected = item["expected_doc_id"]

    results = hybrid_search(query)
    returned_ids = [r["doc_id"] for r in results]

    print("Query:", query)
    print("Returned:", returned_ids)
    print("-" * 60)

    if returned_ids[0] == expected:
        correct_top1 += 1

    if expected in returned_ids:
        correct_topk += 1
        rank = returned_ids.index(expected) + 1
        reciprocal_ranks.append(1.0 / rank)
    else:
        reciprocal_ranks.append(0.0)

total = len(evaluation_set)

print("\n===== HYBRID (FILTERED) =====")
print("Top-1:", correct_top1 / total)
print("Top-3:", correct_topk / total)
print("MRR:", sum(reciprocal_ranks) / total)


# phase5_risk_scoring.py

In [ ]:
import json
import re
from collections import defaultdict

# ----------------------------
# CONFIG
# ----------------------------

METADATA_PATH = "metadata.json"

# ----------------------------
# LOAD METADATA
# ----------------------------

print("Loading metadata...")
with open(METADATA_PATH, "r") as f:
    metadata = json.load(f)

# Keep only table_of_cover docs
metadata = [m for m in metadata if m["doc_type"] == "table_of_cover"]

print(f"Loaded {len(metadata)} table_of_cover documents")

# ----------------------------
# GROUP CHUNKS BY DOCUMENT
# ----------------------------

documents = defaultdict(list)

for m in metadata:
    documents[m["doc_id"]].append(m["text"])

# Merge chunks per document
merged_documents = {
    doc_id: " ".join(texts)
    for doc_id, texts in documents.items()
}

# ----------------------------
# PLAN FEATURE EXTRACTION
# ----------------------------

def extract_plan_features(doc_id, text):
    text_lower = text.lower()

    # Inpatient cover
    has_full_inpatient = (
        "inpatient consultant fees covered" in text_lower
        or "consultants fees covered" in text_lower
        or "fully covered" in text_lower
    )

    # Psychiatric days
    psych_match = re.search(r"(\d+)\s+days", text_lower)
    psychiatric_days = int(psych_match.group(1)) if psych_match else 0

    # Semi-private excess
    excess_match = re.search(r"€\s?(\d+)\s+excess", text_lower)
    semi_private_excess = int(excess_match.group(1)) if excess_match else 0

    return {
        "doc_id": doc_id,
        "has_full_inpatient_cover": has_full_inpatient,
        "psychiatric_days": psychiatric_days,
        "semi_private_excess": semi_private_excess
    }

# Extract features for all plans
plans = []

for doc_id, text in merged_documents.items():
    features = extract_plan_features(doc_id, text)
    plans.append(features)

print("Extracted features for all plans")

# ----------------------------
# PATIENT PROFILE
# ----------------------------

patient_profile = {
    "age": 52,
    "chronic_conditions": ["heart_disease"],
    "medication_frequency": "daily",
    "gp_visits_per_year": 8,
    "specialist_visits_per_year": 6,
    "hospital_admissions_last_2_years": 2,
    "budget_per_month": 100,
    "risk_tolerance": 4
}

# ----------------------------
# RISK SCORING
# ----------------------------

def compute_risk_score(profile):
    score = 0

    if profile["age"] > 60:
        score += 3
    elif profile["age"] > 45:
        score += 2

    score += len(profile["chronic_conditions"]) * 3

    if profile["medication_frequency"] == "daily":
        score += 3
    elif profile["medication_frequency"] == "weekly":
        score += 2

    score += profile["hospital_admissions_last_2_years"] * 2

    if profile["specialist_visits_per_year"] > 5:
        score += 2

    return score


def classify_risk(score):
    if score >= 12:
        return "high"
    elif score >= 6:
        return "medium"
    return "low"


risk_score = compute_risk_score(patient_profile)
risk_level = classify_risk(risk_score)

print(f"Risk score: {risk_score}")
print(f"Risk level: {risk_level}")

# ----------------------------
# PLAN SCORING
# ----------------------------

def score_plan(plan, profile, risk_level):
    score = 0

    if risk_level == "high":
        if plan["has_full_inpatient_cover"]:
            score += 5
        else:
            score -= 5

        if plan["semi_private_excess"] <= 150:
            score += 3

        if plan["psychiatric_days"] >= 100:
            score += 2

    elif risk_level == "medium":
        if plan["has_full_inpatient_cover"]:
            score += 3
        if plan["semi_private_excess"] <= 200:
            score += 2

    else:  # low risk
        score -= plan["semi_private_excess"] / 100

    return score

# Rank plans
ranked_plans = sorted(
    plans,
    key=lambda p: score_plan(p, patient_profile, risk_level),
    reverse=True
)

# ----------------------------
# OUTPUT TOP 3
# ----------------------------

print("\nTop Recommended Plans:\n")

for i, plan in enumerate(ranked_plans[:3], 1):
    print(f"{i}. {plan['doc_id']}")
    print(f"   Full inpatient: {plan['has_full_inpatient_cover']}")
    print(f"   Psychiatric days: {plan['psychiatric_days']}")
    print(f"   Semi-private excess: €{plan['semi_private_excess']}")
    print("-" * 60)


# phase5_risk_scoring_v2.py

Now we stop being generic and become condition-aware + clinically aware + financially aware.

Below is a complete upgraded Phase 5 engine.

This version:

Detects cardiac coverage

Detects high-tech hospital cover

Detects exclusions

Detects chronic illness references

Uses specialist visits

Uses admission history

Uses budget filtering

Scores intelligently

In [ ]:
import json
import re
from collections import defaultdict

# ----------------------------
# CONFIG
# ----------------------------

METADATA_PATH = "metadata.json"

# ----------------------------
# LOAD METADATA
# ----------------------------

print("Loading metadata...")
with open(METADATA_PATH, "r") as f:
    metadata = json.load(f)

metadata = [m for m in metadata if m["doc_type"] == "table_of_cover"]
print(f"Loaded {len(metadata)} table_of_cover documents")

# ----------------------------
# GROUP CHUNKS BY DOCUMENT
# ----------------------------

documents = defaultdict(list)

for m in metadata:
    documents[m["doc_id"]].append(m["text"])

merged_documents = {
    doc_id: " ".join(texts).lower()
    for doc_id, texts in documents.items()
}

# ----------------------------
# PRICE EXTRACTION
# ----------------------------

def extract_price(text):
    # Match €39.02 per month / €39 per month / €39 monthly
    match = re.search(r"€\s?([\d,.]+)\s*(per month|monthly)", text)
    if match:
        return float(match.group(1).replace(",", ""))
    return None

# ----------------------------
# FEATURE EXTRACTION
# ----------------------------

def extract_plan_features(doc_id, text):
    features = {}

    features["doc_id"] = doc_id

    # Inpatient cover
    features["full_inpatient_cover"] = (
        "consultant fees covered" in text
        or "inpatient consultant fees covered" in text
        or "fully covered" in text
    )

    # Cardiac coverage
    features["cardiac_covered"] = (
        "cardiac" in text and "not covered" not in text
    )

    # High-tech hospital
    features["high_tech_access"] = (
        "high-tech hospital" in text and "not covered" not in text
    )

    # Psychiatric days
    psych_match = re.search(r"(\d+)\s+days", text)
    features["psychiatric_days"] = int(psych_match.group(1)) if psych_match else 0

    # Semi-private excess
    excess_match = re.search(r"€\s?(\d+)\s+excess", text)
    features["semi_private_excess"] = int(excess_match.group(1)) if excess_match else 0

    # Exclusions
    features["has_exclusions"] = "not covered" in text

    # Chronic mention
    features["mentions_chronic"] = "chronic" in text

    # Auto price extraction
    features["monthly_price"] = extract_price(text)

    return features

# Extract all plans
plans = []

for doc_id, text in merged_documents.items():
    features = extract_plan_features(doc_id, text)
    plans.append(features)

print("Extracted enhanced plan features.")

# ----------------------------
# PATIENT PROFILE
# ----------------------------

patient_profile = {
    "age": 52,
    "chronic_conditions": ["heart_disease"],
    "medication_frequency": "daily",
    "gp_visits_per_year": 8,
    "specialist_visits_per_year": 6,
    "hospital_admissions_last_2_years": 2,
    "budget_per_month": 100,
    "risk_tolerance": 4
}

# ----------------------------
# RISK COMPUTATION
# ----------------------------

def compute_risk_score(profile):
    score = 0

    if profile["age"] > 60:
        score += 3
    elif profile["age"] > 45:
        score += 2

    score += len(profile["chronic_conditions"]) * 3

    if profile["medication_frequency"] == "daily":
        score += 3

    score += profile["hospital_admissions_last_2_years"] * 2

    if profile["specialist_visits_per_year"] > 5:
        score += 3

    return score


def classify_risk(score):
    if score >= 12:
        return "high"
    elif score >= 6:
        return "medium"
    return "low"


risk_score = compute_risk_score(patient_profile)
risk_level = classify_risk(risk_score)

print(f"Risk score: {risk_score}")
print(f"Risk level: {risk_level}")

# ----------------------------
# PLAN SCORING
# ----------------------------

def score_plan(plan, profile, risk_level):
    score = 0

    # Budget filtering (only if price available)
    if plan["monthly_price"] is not None:
        if plan["monthly_price"] > profile["budget_per_month"]:
            score -= 10
        else:
            score += 2  # reward affordable plan

    # High risk logic
    if risk_level == "high":
        if plan["full_inpatient_cover"]:
            score += 6
        else:
            score -= 6

        if plan["semi_private_excess"] <= 150:
            score += 3

        if plan["psychiatric_days"] >= 100:
            score += 2

    # Condition-aware logic
    if "heart_disease" in profile["chronic_conditions"]:
        if plan["cardiac_covered"]:
            score += 6
        else:
            score -= 5

        if plan["high_tech_access"]:
            score += 4

    # Specialist-heavy
    if profile["specialist_visits_per_year"] > 5:
        if plan["full_inpatient_cover"]:
            score += 2

    # Admission history
    if profile["hospital_admissions_last_2_years"] > 1:
        if plan["semi_private_excess"] <= 100:
            score += 3

    # Penalize exclusions
    if plan["has_exclusions"]:
        score -= 2

    return score

# Rank plans
ranked_plans = sorted(
    plans,
    key=lambda p: score_plan(p, patient_profile, risk_level),
    reverse=True
)

# ----------------------------
# OUTPUT
# ----------------------------

print("\nTop Recommended Plans:\n")

for i, plan in enumerate(ranked_plans[:3], 1):
    print(f"{i}. {plan['doc_id']}")
    print(f"   Price: {plan['monthly_price']}")
    print(f"   Cardiac Covered: {plan['cardiac_covered']}")
    print(f"   High-Tech Access: {plan['high_tech_access']}")
    print(f"   Inpatient Cover: {plan['full_inpatient_cover']}")
    print(f"   Psychiatric Days: {plan['psychiatric_days']}")
    print(f"   Excess: €{plan['semi_private_excess']}")
    print("-" * 60)


In [ ]:
import json
import re
from collections import defaultdict

# ----------------------------
# CONFIG
# ----------------------------

METADATA_PATH = "metadata.json"

# ----------------------------
# LOAD METADATA
# ----------------------------

print("Loading metadata...")
with open(METADATA_PATH, "r") as f:
    metadata = json.load(f)

metadata = [m for m in metadata if m["doc_type"] == "table_of_cover"]
print(f"Loaded {len(metadata)} table_of_cover documents")

# ----------------------------
# GROUP DOCUMENT TEXT
# ----------------------------

documents = defaultdict(list)

for m in metadata:
    documents[m["doc_id"]].append(m["text"])

merged_documents = {
    doc_id: " ".join(texts).lower()
    for doc_id, texts in documents.items()
}

# ----------------------------
# DISEASE RULES ENGINE
# ----------------------------

DISEASE_RULES = {

    "heart_disease": {
        "keywords": ["cardiac", "cardiology", "angioplasty", "stent"],
        "requires_high_tech": True,
        "weight": 7
    },

    "diabetes": {
        "keywords": ["diabetes", "insulin", "endocrinology"],
        "requires_outpatient": True,
        "weight": 6
    },

    "cancer": {
        "keywords": ["oncology", "chemotherapy", "radiotherapy", "oncotype"],
        "requires_high_tech": True,
        "weight": 9
    },

    "psychiatric_disorder": {
        "keywords": ["psychiatric", "mental health", "substance abuse"],
        "requires_psych_cover": True,
        "weight": 6
    },

    "chronic_kidney_disease": {
        "keywords": ["dialysis", "renal"],
        "requires_high_tech": True,
        "weight": 8
    },

    "copd": {
        "keywords": ["respiratory", "pulmonary"],
        "requires_hospital": True,
        "weight": 6
    },

    "autoimmune_disorder": {
        "keywords": ["immunology", "biologic therapy"],
        "requires_specialist": True,
        "weight": 7
    },

    "pregnancy": {
        "keywords": ["maternity", "obstetric"],
        "requires_maternity": True,
        "weight": 8
    },

    "neurological_disorder": {
        "keywords": ["neurology", "mri", "ct scan"],
        "requires_high_tech": True,
        "weight": 7
    },

    "orthopaedic_condition": {
        "keywords": ["orthopaedic", "joint replacement"],
        "requires_hospital": True,
        "weight": 6
    }
}

# ----------------------------
# FEATURE EXTRACTION
# ----------------------------

def extract_plan_features(doc_id, text):

    features = {}
    features["doc_id"] = doc_id

    features["full_inpatient_cover"] = "consultant fees covered" in text or "fully covered" in text
    features["high_tech_access"] = "high-tech hospital" in text and "not covered" not in text
    features["has_psych_cover"] = "psychiatric" in text
    features["has_maternity"] = "maternity" in text
    features["has_exclusions"] = "not covered" in text

    psych_match = re.search(r"(\d+)\s+days", text)
    features["psychiatric_days"] = int(psych_match.group(1)) if psych_match else 0

    excess_match = re.search(r"€\s?(\d+)\s+excess", text)
    features["semi_private_excess"] = int(excess_match.group(1)) if excess_match else 0

    features["text"] = text  # store for keyword matching

    return features


plans = []

for doc_id, text in merged_documents.items():
    plans.append(extract_plan_features(doc_id, text))

print("Extracted plan features.")

# ----------------------------
# PATIENT PROFILE
# ----------------------------

patient_profile = {
    "age": 52,
    "chronic_conditions": ["heart_disease", "diabetes"],
    "medication_frequency": "daily",
    "gp_visits_per_year": 8,
    "specialist_visits_per_year": 6,
    "hospital_admissions_last_2_years": 2,
    "budget_per_month": 100
}

# ----------------------------
# RISK SCORE
# ----------------------------

def compute_risk_score(profile):

    score = 0

    if profile["age"] > 60:
        score += 3
    elif profile["age"] > 45:
        score += 2

    score += len(profile["chronic_conditions"]) * 3

    if profile["medication_frequency"] == "daily":
        score += 3

    score += profile["hospital_admissions_last_2_years"] * 2

    if profile["specialist_visits_per_year"] > 5:
        score += 3

    return score


risk_score = compute_risk_score(patient_profile)

print("Risk Score:", risk_score)

# ----------------------------
# DISEASE-AWARE PLAN SCORING
# ----------------------------

def score_plan(plan, profile):

    score = 0

    # Generic inpatient logic
    if plan["full_inpatient_cover"]:
        score += 4
    else:
        score -= 4

    if plan["semi_private_excess"] <= 150:
        score += 2

    if plan["has_exclusions"]:
        score -= 2

    # Disease-specific logic
    for condition in profile["chronic_conditions"]:

        rules = DISEASE_RULES.get(condition)

        if not rules:
            continue

        # Keyword coverage
        for keyword in rules["keywords"]:
            if keyword in plan["text"]:
                score += rules["weight"]

        # High-tech requirement
        if rules.get("requires_high_tech") and plan["high_tech_access"]:
            score += 3

        # Psychiatric requirement
        if rules.get("requires_psych_cover") and plan["has_psych_cover"]:
            score += 3

        # Maternity requirement
        if rules.get("requires_maternity") and plan["has_maternity"]:
            score += 3

    return score


# Rank plans
ranked_plans = sorted(
    plans,
    key=lambda p: score_plan(p, patient_profile),
    reverse=True
)

# ----------------------------
# OUTPUT
# ----------------------------

print("\nTop Recommended Plans:\n")

for i, plan in enumerate(ranked_plans[:3], 1):
    print(f"{i}. {plan['doc_id']}")
    print(f"   Excess: €{plan['semi_private_excess']}")
    print(f"   High-Tech: {plan['high_tech_access']}")
    print(f"   Psychiatric Cover: {plan['has_psych_cover']}")
    print("-" * 60)


# Phase 5 Pro

In [ ]:
import json
import re
from collections import defaultdict

# ============================
# LOAD METADATA
# ============================

print("Loading metadata...")
with open("metadata.json", "r") as f:
    metadata = json.load(f)

metadata = [m for m in metadata if m["doc_type"] == "table_of_cover"]
print(f"Loaded {len(metadata)} table_of_cover documents")

# ============================
# MERGE TEXT PER PLAN
# ============================

documents = defaultdict(list)

for m in metadata:
    documents[m["doc_id"]].append(m["text"].lower())

merged_documents = {
    doc_id: " ".join(texts)
    for doc_id, texts in documents.items()
}

# ============================
# DISEASE RULES
# ============================

DISEASE_RULES = {

    "heart_disease": {
        "keywords": ["cardiac", "angioplasty", "stent"],
        "weight": 7,
        "requires_high_tech": True
    },

    "diabetes": {
        "keywords": ["diabetes", "insulin", "endocrinology"],
        "weight": 5,
        "requires_outpatient": True
    },

    "cancer": {
        "keywords": ["oncology", "chemotherapy", "radiotherapy", "oncotype"],
        "weight": 9,
        "requires_high_tech": True
    },

    "psychiatric_disorder": {
        "keywords": ["psychiatric", "mental health"],
        "weight": 6,
        "requires_psych": True
    },

    "orthopaedic_condition": {
        "keywords": ["orthopaedic", "joint replacement"],
        "weight": 6,
        "requires_hospital": True
    },

    "neurological_disorder": {
        "keywords": ["neurology", "mri", "ct scan"],
        "weight": 7,
        "requires_high_tech": True
    },

    "respiratory_disease": {
        "keywords": ["respiratory", "pulmonary"],
        "weight": 5
    },

    "chronic_kidney_disease": {
        "keywords": ["dialysis", "renal"],
        "weight": 8,
        "requires_high_tech": True
    },

    "autoimmune_disorder": {
        "keywords": ["immunology", "biologic"],
        "weight": 7
    },

    "pregnancy": {
        "keywords": ["maternity", "obstetric"],
        "weight": 8,
        "requires_maternity": True
    }
}


# ============================
# FEATURE EXTRACTION
# ============================

def extract_plan_features(doc_id, text):

    features = {}
    features["doc_id"] = doc_id
    features["text"] = text

    # Inpatient
    features["full_inpatient_cover"] = (
        "inpatient consultant fees" in text and "covered" in text
    )

    # Psychiatric days
    psych_matches = re.findall(r"(\d+)\s+days", text)
    features["psychiatric_days"] = max([int(x) for x in psych_matches], default=0)

    # Excess
    excess_match = re.search(r"€\s?(\d+)\s+excess", text)
    features["semi_private_excess"] = int(excess_match.group(1)) if excess_match else 0

    # Cardiac copay
    cardiac_match = re.search(r"€\s?(\d{2,5})\s*(?:co-?payment|copayment)[^\n]{0,40}cardiac", text)
    features["cardiac_copay"] = int(cardiac_match.group(1)) if cardiac_match else 0

    # ========================
    # HIGH TECH PARSING
    # ========================

    hightech_match = re.search(
        r"high-tech hospital(.*?)(outpatient|day to day|members benefits|$)",
        text,
        re.DOTALL
    )

    hightech_text = hightech_match.group(1) if hightech_match else ""

    features["high_tech_percent"] = 0
    features["high_tech_restriction"] = 1.0
    features["high_tech_excess"] = 0
    features["high_tech_cardiac_copay"] = 0

    if hightech_text:

        # Extract % cover
        percents = re.findall(r"(\d+)%\s*cover", hightech_text)
        if percents:
            features["high_tech_percent"] = sum(int(p) for p in percents) / len(percents)

        elif "covered" in hightech_text:
            features["high_tech_percent"] = 100

        # Beacon only = restricted network
        if "beacon only" in hightech_text:
            features["high_tech_restriction"] = 0.4

        if "50% cover in mater private" in hightech_text:
            features["high_tech_restriction"] = 0.6

        # High-tech excess
        ht_excess = re.search(r"€\s?(\d+)\s+excess", hightech_text)
        if ht_excess:
            features["high_tech_excess"] = int(ht_excess.group(1))

        # High-tech cardiac copay
        ht_cardiac = re.search(r"€\s?(\d+)[^\n]{0,40}cardiac", hightech_text)
        if ht_cardiac:
            features["high_tech_cardiac_copay"] = int(ht_cardiac.group(1))

    return features


plans = [
    extract_plan_features(doc_id, text)
    for doc_id, text in merged_documents.items()
]

print("Extracted calibrated plan features.")

# ============================
# PATIENT PROFILE
# ============================

patient_profile = {
    "age": 52,
    "chronic_conditions": ["heart_disease", "diabetes"],
    "hospital_admissions_last_2_years": 2,
    "specialist_visits_per_year": 6
}

# ============================
# PLAN SCORING
# ============================

def score_plan(plan, profile):

    score = 0

    # Inpatient
    score += 5 if plan["full_inpatient_cover"] else -5

    # Psychiatric strength
    score += plan["psychiatric_days"] / 40

    # Excess penalty
    score -= plan["semi_private_excess"] / 80

    # Cardiac copay penalty (stronger)
    score -= plan["cardiac_copay"] / 100

    # Disease logic
    for condition in profile["chronic_conditions"]:

        rules = DISEASE_RULES.get(condition)
        if not rules:
            continue

        for keyword in rules["keywords"]:
            if keyword in plan["text"]:
                score += rules["weight"]

        if rules.get("requires_high_tech"):

            # Nonlinear high-tech boost
            ht_score = (
    (plan["high_tech_percent"] / 100) ** 1.5
) * 6



            ht_score *= plan["high_tech_restriction"]

            ht_score -= plan["high_tech_excess"] / 150
            ht_score -= plan["high_tech_cardiac_copay"] / 200

            score += ht_score

    return round(score, 2)


ranked_plans = sorted(
    plans,
    key=lambda p: score_plan(p, patient_profile),
    reverse=True
)

# ============================
# OUTPUT
# ============================

print("\nTop Recommended Plans:\n")

for i, plan in enumerate(ranked_plans[:3], 1):
    print(f"{i}. {plan['doc_id']}")
    print(f"   Score: {score_plan(plan, patient_profile)}")
    print(f"   Excess: €{plan['semi_private_excess']}")
    print(f"   Cardiac Co-pay: €{plan['cardiac_copay']}")
    print(f"   High-Tech %: {plan['high_tech_percent']}")
    print(f"   High-Tech Restriction: {plan['high_tech_restriction']}")
    print(f"   Psychiatric Days: {plan['psychiatric_days']}")
    print("-" * 60)


In [ ]:
import json
import re
from collections import defaultdict

# ============================
# LOAD METADATA
# ============================

print("Loading metadata...")
with open("metadata.json", "r") as f:
    metadata = json.load(f)

metadata = [m for m in metadata if m["doc_type"] == "table_of_cover"]
print(f"Loaded {len(metadata)} table_of_cover documents")

# ============================
# MERGE TEXT PER PLAN
# ============================

documents = defaultdict(list)

for m in metadata:
    documents[m["doc_id"]].append(m["text"].lower())

merged_documents = {
    doc_id: " ".join(texts)
    for doc_id, texts in documents.items()
}

# ============================
# DISEASE RULES
# ============================

DISEASE_RULES = {
    "heart_disease": {
        "keywords": ["cardiac", "angioplasty", "stent"],
        "weight": 7,
        "requires_high_tech": True
    },
    "diabetes": {
        "keywords": ["diabetes", "insulin", "endocrinology"],
        "weight": 5
    },
    "cancer": {
        "keywords": ["oncology", "chemotherapy", "radiotherapy", "oncotype"],
        "weight": 9,
        "requires_high_tech": True
    },
    "psychiatric_disorder": {
        "keywords": ["psychiatric", "mental health"],
        "weight": 6
    },
    "neurological_disorder": {
        "keywords": ["neurology", "mri", "ct scan"],
        "weight": 7,
        "requires_high_tech": True
    },
    "orthopaedic_condition": {
        "keywords": ["orthopaedic", "joint replacement"],
        "weight": 6
    },
    "pregnancy": {
        "keywords": ["maternity", "obstetric"],
        "weight": 8
    }
}

# ============================
# FEATURE EXTRACTION
# ============================

def extract_plan_features(doc_id, text):

    features = {}
    features["doc_id"] = doc_id
    features["text"] = text

    # Inpatient coverage
    features["full_inpatient_cover"] = (
        "inpatient consultant fees" in text and "covered" in text
    )

    # Psychiatric days
    psych_matches = re.findall(r"(\d+)\s+days", text)
    features["psychiatric_days"] = max([int(x) for x in psych_matches], default=0)

    # Semi-private excess
    excess_match = re.search(r"€\s?(\d+)\s+excess", text)
    features["semi_private_excess"] = int(excess_match.group(1)) if excess_match else 0

    # Cardiac copay
    cardiac_match = re.search(
        r"€\s?(\d{2,5})\s*(?:co-?payment|copayment)[^\n]{0,40}cardiac",
        text
    )
    features["cardiac_copay"] = int(cardiac_match.group(1)) if cardiac_match else 0

    # ========================
    # HIGH-TECH PARSING
    # ========================

    hightech_match = re.search(
        r"high-tech hospital(.*?)(outpatient|day to day|members benefits|$)",
        text,
        re.DOTALL
    )

    hightech_text = hightech_match.group(1) if hightech_match else ""

    features["high_tech_percent"] = 0
    features["high_tech_restriction"] = 1.0
    features["high_tech_excess"] = 0
    features["high_tech_cardiac_copay"] = 0

    if hightech_text:

        percents = re.findall(r"(\d+)%\s*cover", hightech_text)

        if percents:
            features["high_tech_percent"] = sum(int(p) for p in percents) / len(percents)
        elif "covered" in hightech_text:
            features["high_tech_percent"] = 100

        if "beacon only" in hightech_text:
            features["high_tech_restriction"] = 0.4

        if "mater private" in hightech_text:
            features["high_tech_restriction"] = min(features["high_tech_restriction"], 0.6)

        ht_excess = re.search(r"€\s?(\d+)\s+excess", hightech_text)
        if ht_excess:
            features["high_tech_excess"] = int(ht_excess.group(1))

        ht_cardiac = re.search(r"€\s?(\d+)[^\n]{0,40}cardiac", hightech_text)
        if ht_cardiac:
            features["high_tech_cardiac_copay"] = int(ht_cardiac.group(1))

    return features


plans = [
    extract_plan_features(doc_id, text)
    for doc_id, text in merged_documents.items()
]

print("Extracted calibrated plan features.")

# ============================
# PATIENT PROFILE
# ============================

patient_profile = {
    "age": 52,
    "chronic_conditions": ["heart_disease", "diabetes"],
    "hospital_admissions_last_2_years": 2,
    "specialist_visits_per_year": 6
}

# ============================
# RISK CLASSIFICATION
# ============================

def compute_risk_score(profile):

    score = 0

    if profile["age"] > 60:
        score += 4
    elif profile["age"] > 45:
        score += 2

    score += len(profile["chronic_conditions"]) * 3
    score += profile["hospital_admissions_last_2_years"] * 2

    if profile["specialist_visits_per_year"] > 5:
        score += 2

    return score


def classify_risk(score):

    if score >= 12:
        return "high"
    elif score >= 6:
        return "medium"
    else:
        return "low"


risk_score = compute_risk_score(patient_profile)
risk_level = classify_risk(risk_score)

print("Risk Score:", risk_score)
print("Risk Level:", risk_level)

# ============================
# SCORING ENGINE
# ============================

def score_plan(plan, profile):

    score = 0

    # Risk scaling
    if risk_level == "high":
        hospital_weight = 1.5
        excess_weight = 1.0
        hightech_weight = 1.5
    elif risk_level == "medium":
        hospital_weight = 1.0
        excess_weight = 1.0
        hightech_weight = 1.0
    else:
        hospital_weight = 0.6
        excess_weight = 1.5
        hightech_weight = 0.6

    # Inpatient
    score += (5 if plan["full_inpatient_cover"] else -5) * hospital_weight

    # Psychiatric differentiation
    psych_boost = min(plan["psychiatric_days"], 150) / 25
    score += psych_boost

    # Excess penalty
    score -= (plan["semi_private_excess"] / 80) * excess_weight

    # Cardiac copay penalty
    score -= plan["cardiac_copay"] / 100

    # Disease logic
    for condition in profile["chronic_conditions"]:

        rules = DISEASE_RULES.get(condition)
        if not rules:
            continue

        for keyword in rules["keywords"]:
            if keyword in plan["text"]:
                score += rules["weight"]

        if rules.get("requires_high_tech"):

            ht_score = (
                (plan["high_tech_percent"] / 100) ** 1.5
            ) * 6

            ht_score *= plan["high_tech_restriction"]
            ht_score *= hightech_weight

            ht_score -= plan["high_tech_excess"] / 150
            ht_score -= plan["high_tech_cardiac_copay"] / 200

            score += ht_score

    # Over-insurance penalty for low risk
    if risk_level == "low":

        if plan["high_tech_percent"] > 80:
            score -= 2

        if plan["psychiatric_days"] > 120:
            score -= 2

    return round(score, 2)


ranked_plans = sorted(
    plans,
    key=lambda p: score_plan(p, patient_profile),
    reverse=True
)

# ============================
# OUTPUT
# ============================

print("\nTop Recommended Plans:\n")

for i, plan in enumerate(ranked_plans[:3], 1):
    print(f"{i}. {plan['doc_id']}")
    print(f"   Score: {score_plan(plan, patient_profile)}")
    print(f"   Excess: €{plan['semi_private_excess']}")
    print(f"   Cardiac Co-pay: €{plan['cardiac_copay']}")
    print(f"   High-Tech %: {plan['high_tech_percent']}")
    print(f"   High-Tech Restriction: {plan['high_tech_restriction']}")
    print(f"   Psychiatric Days: {plan['psychiatric_days']}")
    print("-" * 60)


In [ ]:
# ============================
# SCENARIO EVALUATION
# ============================

SCENARIOS = [

    {
        "name": "High-Risk Cardiac + Diabetes",
        "profile": {
            "age": 55,
            "chronic_conditions": ["heart_disease", "diabetes"],
            "medication_frequency": "daily",
            "specialist_visits_per_year": 6,
            "hospital_admissions_last_2_years": 2
        }
    },

    {
        "name": "Cancer Patient",
        "profile": {
            "age": 48,
            "chronic_conditions": ["cancer"],
            "medication_frequency": "daily",
            "specialist_visits_per_year": 8,
            "hospital_admissions_last_2_years": 3
        }
    },

    {
        "name": "Maternity Case",
        "profile": {
            "age": 32,
            "chronic_conditions": ["pregnancy"],
            "medication_frequency": "monthly",
            "specialist_visits_per_year": 4,
            "hospital_admissions_last_2_years": 0
        }
    },

    {
        "name": "Psychiatric Condition",
        "profile": {
            "age": 40,
            "chronic_conditions": ["psychiatric_disorder"],
            "medication_frequency": "daily",
            "specialist_visits_per_year": 5,
            "hospital_admissions_last_2_years": 1
        }
    },

    {
        "name": "Young Low-Risk Adult",
        "profile": {
            "age": 25,
            "chronic_conditions": [],
            "medication_frequency": "none",
            "specialist_visits_per_year": 1,
            "hospital_admissions_last_2_years": 0
        }
    },

    {
        "name": "Neurological Disorder",
        "profile": {
            "age": 60,
            "chronic_conditions": ["neurological_disorder"],
            "medication_frequency": "daily",
            "specialist_visits_per_year": 7,
            "hospital_admissions_last_2_years": 2
        }
    }
]

print("\n==============================")
print("SCENARIO EVALUATION")
print("==============================")

for scenario in SCENARIOS:

    name = scenario["name"]
    profile = scenario["profile"]

    ranked = sorted(
        plans,
        key=lambda p: score_plan(p, profile),
        reverse=True
    )

    print(f"\n--- Scenario: {name} ---")

    for i, plan in enumerate(ranked[:3], 1):
        print(f"{i}. {plan['doc_id']}")
        print(f"   Score: {score_plan(plan, profile)}")
        print(f"   Excess: €{plan['semi_private_excess']}")
        print(f"   High-Tech %: {plan['high_tech_percent']}")
        print(f"   Psychiatric Days: {plan['psychiatric_days']}")
        print("-" * 50)


# Phase 6

In [ ]:
! pip install transformers accelerate bitsandbytes rank_bm25

In [ ]:
import torch
import gc
import os

# Delete variables if they exist
for var in ["model", "tokenizer"]:
    if var in globals():
        del globals()[var]

gc.collect()
torch.cuda.empty_cache()

# Force defragment
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print("GPU cleaned.")
print("Allocated:", torch.cuda.memory_allocated() / 1e9, "GB")
print("Reserved :", torch.cuda.memory_reserved() / 1e9, "GB")


In [ ]:
import json
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# =====================================================
# CONFIG
# =====================================================

TOP_K_PLANS = 3
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"

# =====================================================
# RISK LEVEL LABEL
# =====================================================

def get_risk_label(risk_score):
    if risk_score >= 14:
        return "HIGH"
    elif risk_score >= 8:
        return "MEDIUM"
    else:
        return "LOW"

risk_level = get_risk_label(risk_score)

# =====================================================
# RANK PLANS
# =====================================================

ranked_plans = sorted(
    plans,
    key=lambda p: score_plan(p, patient_profile),
    reverse=True
)

top_plans = ranked_plans[:TOP_K_PLANS]

# =====================================================
# STRUCTURED EVIDENCE BUILDER
# =====================================================

def build_structured_evidence(plan, profile):

    evidence = {}

    text = plan.get("text", "")

    # Inpatient
    if plan.get("full_inpatient", False):
        evidence["inpatient"] = "Inpatient consultant fees and scans are covered."
    else:
        evidence["inpatient"] = "Inpatient consultant fees are not fully covered."

    # Excess
    excess = plan.get("excess", 0)
    evidence["excess"] = f"Plan excess is €{excess}."

    # Disease relevance
    disease_evidence = []

    for condition in profile["chronic_conditions"]:
        rules = DISEASE_RULES.get(condition)
        if not rules:
            continue

        matched = False
        for kw in rules["keywords"]:
            if kw in text:
                disease_evidence.append(
                    f"Mentions '{kw}' related to {condition}."
                )
                matched = True

        if not matched:
            disease_evidence.append(
                f"No explicit mention of {condition} coverage."
            )

    evidence["disease"] = disease_evidence

    # High-tech
    if plan.get("high_tech", False):
        evidence["hightech"] = "High-tech hospital coverage available."
    else:
        evidence["hightech"] = "High-tech hospital coverage not available or restricted."

    return evidence

# =====================================================
# LOAD MODEL (4-bit)
# =====================================================

print("Loading LLM...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

print("LLM loaded.")

# =====================================================
# EXPLANATION GENERATOR
# =====================================================

def generate_explanation(rank, plan, next_plan=None):

    breakdown, total_score = compute_score_breakdown(plan, patient_profile)
    evidence = build_structured_evidence(plan, patient_profile)

    comparison_line = ""
    if next_plan:
        comparison_line = (
            f"This plan ranked above {next_plan['doc_id']} "
            f"because it achieved a higher total score."
        )

    prompt = f"""
You are a medical insurance ranking explanation engine.

RULES:
- Use ONLY the structured evidence provided.
- Do NOT invent benefits.
- Connect patient conditions to coverage.
- Explain ranking logically.
- Be concise and factual.
- Max 180 words.

PATIENT PROFILE:
{json.dumps(patient_profile)}

RISK LEVEL: {risk_level}

PLAN RANK: #{rank}
PLAN ID: {plan["doc_id"]}
TOTAL SCORE: {total_score}

SCORE BREAKDOWN:
{json.dumps(breakdown)}

STRUCTURED EVIDENCE:
{json.dumps(evidence, indent=2)}

TASK:
Explain why this plan ranked #{rank}.
Link:
- inpatient component
- excess penalty
- disease match
- high-tech impact
Mention limitations clearly.
{comparison_line}
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=220,
        temperature=0.0,
        do_sample=False
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# =====================================================
# RUN EXPLANATIONS
# =====================================================

print("\n==============================")
print("DECISION-AWARE EXPLANATIONS")
print("==============================")

for i, plan in enumerate(top_plans):

    next_plan = top_plans[i+1] if i+1 < len(top_plans) else None

    explanation = generate_explanation(
        rank=i+1,
        plan=plan,
        next_plan=next_plan
    )

    print("\n====================================")
    print("PLAN:", plan["doc_id"])
    print("====================================")
    print(explanation)
